In [ ]:
Baseline Model Training & Selection

In [1]:
import sys 
sys.path.append("../")
sys.path.append("../../../")

In [2]:
from src.models import train_model
import pandas
import pickle
import os
from baseline_requirements import metrics 
from text_classification import text

import warnings 

warnings.filterwarnings('ignore')

Loading raw dataset

In [3]:
training_set = pandas.read_csv("../data/processed_data/training_set.csv")
validation_set = pandas.read_csv("../data/processed_data/validation_set.csv")
testing_set = pandas.read_csv("../data/processed_data/testing_set.csv")

Choosing the best baseline model

In [4]:
from sklearn.linear_model import LogisticRegression 
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC
from sklearn.metrics import make_scorer
import numpy

In [5]:
models = {
    "lg": LogisticRegression(),
    "mnb": MultinomialNB(),
    "svm": LinearSVC(),
}

Feature Selection using Training Set

In [ ]:
from sklearn.feature_selection import RFECV
from sklearn.metrics import recall_score
from sklearn.model_selection import StratifiedKFold

def balanced_recall(y_true, y_pred):
    return recall_score(y_true, y_pred, average='weighted')


metric = make_scorer(balanced_recall, greater_is_better=True)

feature_importances = {}

X_train = training_set.drop(columns=['category'])
Y_train = training_set['category']

for model_name, model in models.items():
    # Computing Feature importances using Recursive Feature Elimination (RFE)
    cv_results = RFECV(
        estimator=model,
        step=2,
        min_features_to_select=4,
        cv=StratifiedKFold(n_splits=5),
        n_jobs=-1,
        scoring=metric,
    )
    print('fitting %s model')
    cv_results.fit(X_train, Y_train)
    print('settings importances for model %s')
    # storing output important features 
    feature_importances[model_name] = {
        'important_features': cv_results.cv_results_
    }

feature_importances

: 

Hyperparameter Tuning using Validation Set

In [ ]:
hyperparams = {
    "lg": {
        'penalty' : ['l1', 'l2', 'elasticnet', 'none'], # type of regularization
        'C' : numpy.logspace(-4, 4, 20),  # C parameter
        'solver' : ['lbfgs','newton-cg','liblinear','sag','saga'],
        'max_iter' : [100, 1000,2500, 5000] # maximum number of iterations
    },
    "mnb": {
        'alpha': [0.1, 0.5, 1.0, 1.5], # alpha regularization parameter
        'fit_prior': [True, False]
    },
    "svm": {
        'C': 1.0,  # Regularization strength
        'loss': 'squared_hinge',  # Loss function
        'dual': [True, False],  # Whether to solve the dual or primal problem
        'fit_intercept': [True, False],  # Whether to calculate the intercept
        'max_iter': 1000,  # Maximum number of iterations for the solver
    }
}

In [ ]:
output = {}

for model_name, model in models.items():
    score, best_model = train_model.fine_tune_model(
        k_cross=5,
        training_set=validation_set,
        target_variable="category",
        hyperparams=hyperparams[model_name],
        model=model,
        loss_function_or_scorer_metric=metric,
    )
    output[model_name] = {
        "best_model": best_model,
        "best_score": score
    }

Visualizing output and choosing best baseline model

In [ ]:
output

Picking the best model, based on a given score from the HP output

In [ ]:
chosen_model = sorted(output, lambda model: model['best_score'], reverse=True)[0]

Testing model using cross-validation on Testing set

In [ ]:
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_validate

X, Y = testing_set.drop(columns=['category']), testing_set['category']
cv = cross_validate(
    estimator=chosen_model,
    scoring=metric,
    X=testing_set,
    cv=StratifiedKFold(n_splits=5, shuffle=True)
)
print('test metric score: %s' % cv['test_score'])

Estimating model performance according to baseline metrics

In [ ]:
print('meets expected metric: ', metrics.AVERAGED_WEIGHTED_RECALL <= cv['test_score'])

Saving baseline model

In [ ]:
pickle.dump(chosen_model, open('../models/baseline_classifier.pkl', mode='wb'))